In [ ]:
import numpy as np

# Solving equations by Gaussian elimination

We want to solve a system of linear equations, which we assume are represented using the following Matrix equation:
$$ Ax = b $$

where $A$ is a square matrix of size $N\times N$ and $b$ is a column vector of size $N\times 1$.  There are $N$ unknowns $x_1, x_2 \ldots x_N$ that you want to solve for.

It is assumed you already know how the process works, but to refresh your memory, you could use the reference material at [LibreTexts](https://math.libretexts.org/Bookshelves/Algebra/Book%3A_Algebra_and_Trigonometry_(OpenStax)/11%3A_Systems_of_Equations_and_Inequalities/11.06%3A_Solving_Systems_with_Gaussian_Elimination).

Basically it involves making the A matrix *triangular* and ultimately into the shape of an identity matrix.

In [ ]:
# Input matrices - the set of equations - 2 variables x1 and x2
A = [ [2,3], [1,-1] ]
B = [6,1/2]
print(A)
print(B)

In [ ]:
# Normalize row 1
norm = A[0][0]
for i in range(len(A[0])): A[0][i] /= norm
B[0] = B[0]/norm
print(A)
print(B)

In [ ]:
# Eliminate row 2 - A[1] - need to check and ensure div-by-zero etc doesnt happen
norm = A[1][0] / A[0][0]
for i in range(len(A[1])): A[1][i] = A[1][i] - A[0][i] * norm
B[1] = B[1] - B[0] * norm
print(A)
print(B)

In [ ]:
# Normalize row 2 - B[1] will now contain the solution for x2
norm = A[1][1]
for i in range(len(A[1])): A[1][i] = A[1][i] / norm
B[1] = B[1] / norm
print(A)
print(B)

In [ ]:
# Sub back and solve for B[0] <-> x1
# This can be seen as eliminating A[0][1]
norm = A[0][1] / A[0][0]
# note that len(A) will give number of rows
for i in range(len(A)): 
    A[i][1] = A[i][1] - A[i][0] * norm
    B[i] = B[i] - A[i][0] * norm
print(A)
print(B)

## Problems with Gaussian elimination

There are several obvious problems with the method outlined here.  These include:

- Performance - Python lists are not the most efficient way to store matrices
- Zeros: the simple example does not consider a scenario where one of the values on the diagonal may be 0.  Then some shuffling of rows is required.
- Numerical stability: there are several *normalization* steps involved, where it is quite possible for the values to blow up out of control if not managed properly.  Usually some kind of pivoting techniques are used to get around these issues.

In [ ]:
import numpy as np
A1 = np.array( [ [2,3], [1,-1] ] )
B1 = np.array( [6, 1/2] )
np.linalg.solve(A1, B1)

# SPICE basics

Our goal is to implement a SPICE simulator.  In order to do this, we first need to read in the circuit description from a text file.  To start with, we will only consider the basic elements of SPICE: Voltage sources, Current sources, and Resistors.  A typical SPICE netlist would look like this:

```spice
.circuit
V1   1 GND  dc 2
R1   1   2     1    
R2   2 GND     1  
.end
```

This is basically a *netlist* with 3 *nodes* - one of which is Ground (GND) which is assumed to be have a voltage of 0V.  We can write down Kirchhoff's current law (KCL) equations at each node, to account for current balance.  In addition, we will have some equations that specify the voltages between nodes having a direct voltage source, since there is no resistance there to provide an equation.

For the above example, the equations will be

### Current balance
$$
\begin{aligned}
I_1               & + & \frac{V_1-V_2}{R_1}     & = & 0 \\
\frac{V_2-0}{R_2} & + & \frac{V_2-V_1}{R_1}     & = & 0 
\end{aligned}
$$

### Voltage
$$
\begin{aligned}
V_1 & - & 0 & = & 2
\end{aligned}
$$

which can be written in Matrix form as:

$$
\begin{bmatrix}
\frac{1}{R_1}   & \frac{-1}{R_1}              & 1 \\
\frac{-1}{R_1}  & \frac{1}{R_1}+\frac{1}{R_2} & 0 \\
1  & 0  & 0
\end{bmatrix}
\begin{bmatrix}
V_1 \\
V_2 \\
I_1
\end{bmatrix}
=
\begin{bmatrix}
0 \\
0 \\
2
\end{bmatrix}
$$

At this point, you have reduced the solving of the SPICE equations to a known problem (linear equation solving) that you already know how to do.

# String and File manipulation

Given a SPICE netlist like the one above, you need to *read* it and construct an internal matrix as described above.  For string manipulation, there are a few helpful utility functions that we can see here.

circ = """.circuit
R1 GND 1 1  
R2 1 2 1    
V1 GND 2 dc 2
.end
""".splitlines()
for l in circ:
    if l[0] == 'R':
        print("Found a resistor")
    elif l[0] == 'V':
        print("Found a voltage source with value: ", float(l.split()[4]))